# Defining Release Scenarios

The whole point of `GasDispersion.jl` is to model chemical releases into the atmosphere, to do this one needs to have a defined chemical release scenario. 

## A Note on the Coordinate System

In general releases are assumed to originate from a single point, with the coordinate system is such that the release point is at *x=0*, *y=0*, *z=h* where *x* is the downwind distance, *y* the cross wind distance, and *z* the elevation and *h* is the height of the release point.

<a title="Mbeychok, CC BY-SA 3.0 &lt;http://creativecommons.org/licenses/by-sa/3.0/&gt;, via Wikimedia Commons" href="https://commons.wikimedia.org/wiki/File:Gaussian_Plume.png"><img alt="Gaussian Plume" src="https://upload.wikimedia.org/wikipedia/commons/1/10/Gaussian_Plume.png"></a>

<a href="https://commons.wikimedia.org/wiki/File:Gaussian_Plume.png">Mbeychok</a>, <a href="http://creativecommons.org/licenses/by-sa/3.0/">CC BY-SA 3.0</a>, via Wikimedia Commons

## Defining a Scenario

This scenario is adapted from CCPS *Guidelines for Consequence Analysis of Chemical Releases*, CCPS, pg 40.

Suppose we are evaluating the consequences of a leak from a propane storage tank. For the purposes of this suppose it is liquid propane. The leak is from a 10mm hole situated 1m above the ground and the tank head-space is pressurized to 0.1barg with 2m of liquid propane above the hole.

To start with we define the `Substance` with the [properties of propane](https://webbook.nist.gov/cgi/inchi/InChI%3D1S/C3H8/c1-3-2/h3H2%2C1-2H3)

In [1]:
using GasDispersion

In [2]:
propane = Substance(name = :propane,
                    gas_density = 2.6371,          # Propane, NIST Webbook
                    liquid_density = 578.38,       # Propane, NIST Webbook
                    reference_temp= 233.19,
                    reference_pressure= 111325.0,
                    boiling_temp = 233.19,         # Propane, NIST Webbook
                    latent_heat = 16.25/44.0956,   # Propane, NIST Webbook
                    gas_heat_capacity = 1.4528,    # Propane, NIST Webbook
                    liquid_heat_capacity = 2.2560) # Propane, NIST Webbook

Substance: propane 


Suppose we have already worked out that:
- the liquid is at the boiling point, 233.19 K
- the pressure after the release is ambient, 101325 Pa
- the liquid is jetting out at velocity of 5.41 m/s,
- the mass emission rate of 0.246 kg/s, 
- the jet is just pure liquid. 

We have enough information to define the *release*

In [3]:
rel = Release(mass_rate = 0.246, # kg/s
              duration = Inf, # assuming a continuous release
              diameter = 0.01, # m
              velocity = 5.41, # m/s
              height = 1.0, # m
              pressure = 101325, # Pa
              temperature = 233.19, # K
              fraction_liquid = 1.0)

Release conditions:
    ṁ: 0.246 kg/s 
    Δt: Inf s 
    d: 0.01 m 
    u: 5.41 m/s 
    h: 1.0 m 
    P: 101325 Pa 
    T: 233.19 K 
    f_l: 1.0  


Now we need to define the `Atmosphere` into which the `Substance` is being `Release`d, the default assumption is a simple model of dry air (zero humidity) which acts like an ideal gas, at ambient conditions (25 C and 1 atm), with windspeed of 1.5 m/s (measured at 10 m) and a class F atmospheric stability.

In [4]:
# this is equivalent to DryAir()
atm = DryAir(pressure = 101325, # Pa
             temperature = 298.15, # K
             windspeed = 1.5, # m/s
             stability = ClassF) # can be ClassA, ..., ClassF

Atmospheric conditions:
    P: 101325 Pa 
    T: 298.15 K 
    Rs: 287.0500676 J/kg/K 
    u: 1.5 m/s 
    h: 10 m 
    stability: ClassF  


The `Scenario` is then the combination of a substance, release, and atmosphere:

In [5]:
scn = Scenario(substance = propane,
               release = rel,
               atmosphere = atm)

Substance: propane 
Release conditions:
    ṁ: 0.246 kg/s 
    Δt: Inf s 
    d: 0.01 m 
    u: 5.41 m/s 
    h: 1.0 m 
    P: 101325 Pa 
    T: 233.19 K 
    f_l: 1.0  
Atmospheric conditions:
    P: 101325 Pa 
    T: 298.15 K 
    Rs: 287.0500676 J/kg/K 
    u: 1.5 m/s 
    h: 10 m 
    stability: ClassF  


`GasDispersion` also comes with some basic tools for constructing simple release scenarios for common situations

## Hole Discharge

Defining scenarios by working out all of the details yourself can get rather tedious. Instead we could have used a simple jet model assuming a circular hole of negligible length.

### Liquid Jet

In this case we are assuming the jet is purely a liquid. First we need to calculate the pressure at the hole, which is simply the pressure in the tank head space plus the liquid head above the hole.

In [6]:
P_atm = 101325 # Pa
P_head = 0.1e5 # 0.1 barg

g  = 9.80616 # m/s^2, gravitational acceleration
ρₗ = 578.38 # kg/m^3, liquid propane
hₗ = 2 # m, height of liquid above the hole

P1_gauge = P_head + ρₗ*g*hₗ
P1 = P1_gauge + P_atm

122668.3736416

Then we can use `scenario_builder` to generate a `Scenario` for us, given a substance and the atmosphere.

The remaining parameters of the release are calculated assuming the Bernoulli equation for a liquid jet

$$u_j = c_d  \sqrt{ 2 \left( P_1 - P_2 \over \rho_l \right) }$$

where the hole is assumed to be circular, with a default discharge coefficient of 0.63.

In [7]:
scn = scenario_builder(propane, JetSource, atm; 
       phase = :liquid, # can be either :liquid or :gas 
       diameter = 0.01, # m
       pressure = P1, # Pa
       temperature = 233.19, # K
       height = 1.0) # m, height of hole above the ground

Substance: propane 
Release conditions:
    ṁ: 0.24585757618480786 kg/s 
    Δt: Inf s 
    d: 0.01 m 
    u: 5.412282381327767 m/s 
    h: 1.0 m 
    P: 101325 Pa 
    T: 233.19 K 
    f_l: 1.0  
Atmospheric conditions:
    P: 101325 Pa 
    T: 298.15 K 
    Rs: 287.0500676 J/kg/K 
    u: 1.5 m/s 
    h: 10 m 
    stability: ClassF  


If you are content to use the default atmosphere, this can be simplified to

In [8]:
scn = scenario_builder(propane, JetSource; 
       phase = :liquid, # can be either :liquid or :gas 
       diameter = 0.01, # m
       pressure = P1, # Pa
       temperature = 233.19, # K
       height = 1.0) # m, height of hole above the ground

Substance: propane 
Release conditions:
    ṁ: 0.24585757618480786 kg/s 
    Δt: Inf s 
    d: 0.01 m 
    u: 5.412282381327767 m/s 
    h: 1.0 m 
    P: 101325 Pa 
    T: 233.19 K 
    f_l: 1.0  
Atmospheric conditions:
    P: 101325 Pa 
    T: 298.15 K 
    Rs: 287.0500676 J/kg/K 
    u: 1.5 m/s 
    h: 10 m 
    stability: ClassF  


### Gas Phase

Suppose instead that we are interested in a leak from the top of the tank, say at a height of 3.5 m, where the jet will be pure gas. A simple model for an isentropic nozzle using an ideal gas can be used to model the release.

In this case the mass velocity is calculated assuming a modified Bernoulli equation, which for non-choked flow is:

$$ G = \rho u = c_d \sqrt{ \rho_1 P_1 \left( 2 k \over k-1 \right) \left[ \left(P_2 \over P_1\right)^{2 \over k} - \left(P_2 \over P_1\right)^{k+1 \over k} \right]} $$

and for choked flow is:

$$ G = c_d \sqrt{ \rho_1 P_1 k \left( 2 \over k+1 \right)^{k+1 \over k-1} } $$

where flow is choked when:

$$ \left(P_2 \over P_1 \right) \lt \left( 2 \over k+1 \right)^{k \over k-1} $$

The relations for isentropic flow of an ideal gas are used to calculate the release temperature, and pressure as well.

The model similarly assumes that the hole is circular and uses a default discharge coefficient of 0.63 (which can be overridden if you wish)

In [9]:
scn = scenario_builder(propane, JetSource; 
       phase = :gas, # can be either :liquid or :gas 
       diameter = 0.01, # m
       pressure = P_head+P_atm, # Pa
       temperature = 233.19, # K
       height = 3.5) # m, height of hole above the ground

Substance: propane 
Release conditions:
    ṁ: 0.01080282197060049 kg/s 
    Δt: Inf s 
    d: 0.01 m 
    u: 55.785065462005136 m/s 
    h: 3.5 m 
    P: 101325.0 Pa 
    T: 227.00270856767614 K 
    f_l: 0.0  
Atmospheric conditions:
    P: 101325 Pa 
    T: 298.15 K 
    Rs: 287.0500676 J/kg/K 
    u: 1.5 m/s 
    h: 10 m 
    stability: ClassF  


At this point the scenario can be passed to the `plume` function to generate a solution with the specified dispersion model